# 重构

- 一个影片出租的例子, 计算每位顾客消费金额, 并打印详单
- 顾客: 租了那些影片, 租期多长, 常客积分(根据片种和是否为新片而不同)
- 影片: 分类(儿童,普通, 新片)

In [3]:
# 电影类
class Movie(object):
    CHILDREN = 2
    REGULAR = 0
    NEW_RELEASE = 1
    def __init__(self, title, priceCode):
        self._title = title
        self._priceCode = priceCode
    def getPriceCode(self):
        return self._priceCode
    def setPriceCode(self, arg):
        self._priceCode = arg
    def getTitle(self):
        return self._title
    # 使用property方便后续调整读取属性逻辑； 删除是因为在类的命名空间两个函数已经没用了，参照flask源码
    PriceCode = property(getPriceCode, setPriceCode)
    del getPriceCode, setPriceCode
# 租赁行为类
class Rental(object):
    def __init__(self, movie, daysRented):
        self._movie = movie
        self._daysRented = daysRented
    # In python, 这两个get方法是没必要的
    def getDaysRented(self):
        return self._daysRented
    def getMovie(self):
        return self._movie
    
# 顾客类
class Customer(object):
    
    def __init__(self, name):
        self._name = name
        self._rentals = []
        
    def addRental(self, arg):
        self._rentals.append(arg)
        
    def getName(self):
        return self._name
    
    def statement(self):
        totalAmount = 0
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            thisAmount = 0
            if rental.getMovie().getPriceCode() == 0:
                thisAmount += 2
                if rental.getDaysRented() > 2:
                    thisAmount += (rental.getDaysRented() - 2) * 1.5
            elif rental.getMovie().getPriceCode() == 1:
                thisAmount += rental.getDaysRented() * 3
            else:
                thisAmount += 1.5
                if rental.getDaysRented() > 3:
                    thisAmount += (rental.getDaysRented() - 3) * 1.5
            # add frequent renter points
            frequentRenterPoints += 1
            # add bonus for a two day new release rental
            if (rental.getMovie().getPriceCode() == 1) and (rental.getDaysRented() > 1):
                frequentRenterPoints += 1
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(thisAmount) + "\n"
            totalAmount += thisAmount
        # add footer lines
        result += "Amount owed is " + str(totalAmount) + "\n"
        result += "You earned " + str(frequentRenterPoints) + " frequent renter points"
        return result

### 需求调整

- 希望增加HTML输出详单
    - 无法复用 `statement()`, 重写一个`HTMLStatement()`, 大量复制`statement()`行为
    - 计费标准调整: 同时修改`statement()`和`HTMLStatement()`, 如果程序可能需要修改, 复制粘贴行为存在潜在威胁
    - 改变影片分类, 影响消费和常客积分计算方式, 

> 如果你为程序添加一个特性, 而代码结构无法方便的达成目的, 先重构那个程序, 是特性的添加比较容易, 再添加特性

### 重构的第一步

- 建立一套可靠的测试环境, 避免引入BUG
    - 先假设一些顾客, 各自租几个不同影片, 然后产生报表,并和参考报表比较
    - 测试必须要有自我检查能力
    
### 分解并重组`statement()`

- 代码越小, 功能越容易管理
- 把长函数切成小块代码, 并移至更合适的类, 以降低重复量


#### **Extract Method**
    - `if..elif..else` 语句
    - 首先找到函数内部的局部变量和参数, `rental`和`thisAmount`, 前者未被修改, 后者会被修改, __任何不会被修改的变量都可以被当成参数传入新的函数__, 会被修改的变量就需要格外小心.如果只有一个变量会被修改, 可以当成返回值
    - `thisAmount` 在每次循环起始处设置为0, 在if之前不会改变, 可以直接把新函数的返回值赋值给它

In [13]:
# 重构后的

class Customer(object):
    # 省略
    def statement(self):
        totalAmount = 0
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            thisAmount = 0
            thisAmount = amountFor(rental)
            # add frequent renter points
            frequentRenterPoints += 1
            # add bonus for a two day new release rental
            if (rental.getMovie().getPriceCode() == 1) and (rental.getDaysRented() > 1):
                frequentRenterPoints += 1
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(thisAmount) + "\n"
            totalAmount += thisAmount
        # add footer lines
        result += "Amount owed is " + str(totalAmount) + "\n"
        result += "You earned " + str(frequentRenterPoints) + " frequent renter points"
        return result
    # 注意 返回值类型
    def amountFor(self, rental):
        thisAmount = 0
        if rental.getMovie().getPriceCode() == 0:
            thisAmount += 2
            if rental.getDaysRented() > 2:
                thisAmount += (rental.getDaysRented() - 2) * 1.5
        elif rental.getMovie().getPriceCode() == 1:
            thisAmount += rental.getDaysRented() * 3
        else:
            thisAmount += 1.5
            if rental.getDaysRented() > 3:
                thisAmount += (rental.getDaysRented() - 3) * 1.5
        return thisAmount
# TODO: 看一下python extract method工具$chufengze$2018年11月15日$

- 更进一步, 修改变量名称
    - 唯有写出人类容易理解的代码,才是优秀的程序员
- 代码应该表现自己的目的

In [14]:
# 修改变量名
def amountFor(rental):
    result = 0
    if rental.getMovie().getPriceCode() == 0:
        result += 2
        if rental.getDaysRented() > 2:
            result += (rental.getDaysRented() - 2) * 1.5
    elif rental.getMovie().getPriceCode() == 1:
        result += rental.getDaysRented() * 3
    else:
        result += 1.5
        if rental.getDaysRented() > 3:
            result += (rental.getDaysRented() - 3) * 1.5
    return result

- `amountFor()`使用了`Rental`类的信息, 并没有使用`Customer`的信息
- **函数应该放在它所使用的数据所属对象内**, 所以`amountFor()`应该移动到`Rental`类内
- 有时候也会保留旧函数，让它调用新函数（如果这个方法是一个接口， 又不想调整其他类的接口）


#### **Move Method**

In [2]:
class Rental(object):
    def getCharge(self):
        result = 0
        if self.getMovie().getPriceCode() == 0:
            result += 2
            if self.getDaysRented() > 2:
                result += (self.getDaysRented() - 2) * 1.5
        elif self.getMovie().getPriceCode() == 1:
            result += self.getDaysRented() * 3
        else:
            result += 1.5
            if self.getDaysRented() > 3:
                result += (self.getDaysRented() - 3) * 1.5
        return result
# 
class Customer(object):

    def statement(self):
        totalAmount = 0
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            thisAmount = 0
            thisAmount = rental.getCharge()
            # add frequent renter points
            frequentRenterPoints += 1
            # add bonus for a two day new release rental
            if (rental.getMovie().getPriceCode() == 1) and (rental.getDaysRented() > 1):
                frequentRenterPoints += 1
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(thisAmount) + "\n"
            totalAmount += thisAmount
        # add footer lines
        result += "Amount owed is " + str(totalAmount) + "\n"
        result += "You earned " + str(frequentRenterPoints) + " frequent renter points"
        return result

- `thisAmount` 多余了，他接受`rental.getCharge()`返回结果后没有发生任何改变
- 使用 **Replace Temp with Query** 去除 `thisAmount`

> 临时变量太多不容易跟踪， 但有可能因此付出性能的代价， 如此例子之中`getCharge`被计算了两次，但是可以在`Rental`类中进行优化

In [4]:
class Customer(object):

    def statement(self):
        totalAmount = 0
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            # add frequent renter points
            frequentRenterPoints += 1
            # add bonus for a two day new release rental
            if (rental.getMovie().getPriceCode() == 1) and (rental.getDaysRented() > 1):
                frequentRenterPoints += 1
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(rental.getCharge()) + "\n"
            totalAmount += rental.getCharge()
        # add footer lines
        result += "Amount owed is " + str(totalAmount) + "\n"
        result += "You earned " + str(frequentRenterPoints) + " frequent renter points"
        return result

#### 提炼常客积分代码

- 同样`Extract Method`，`rental`作为参数传入，一个临时变量`frequentRenterPoints`， 先有初始值，但是在该段逻辑并未读取，所以只需把提取的函数结果累加上去就可以了
- 同样该段逻辑与`rental`相关， 搬运出去

In [6]:
class Customer(object):

    def statement(self):
        totalAmount = 0
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            # add frequent renter points
            frequentRenterPoints += rental.getFrequentRenterPoints()
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(rental.getCharge()) + "\n"
            totalAmount += rental.getCharge()
        # add footer lines
        result += "Amount owed is " + str(totalAmount) + "\n"
        result += "You earned " + str(frequentRenterPoints) + " frequent renter points"
        return result
class Rental(object):
    
    def getFrequentRenterPoints(self):
        if (getMovie().getPriceCode() == Movie.NEW_RELEASE) \
            and (getDaysRented() > 1):
            return 2
        else:
            return 1     

- `frequentRenterPoints` 和 `totalAmount` 一样都是从相关对象获得某个总量，其他函数可能需要这些方法

In [8]:
class Customer(object):
    
    def __init__(self, name):
        self._name = name
        self._rentals = []
        
    def statement(self):
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            # add frequent renter points
            frequentRenterPoints += rental.getFrequentRenterPoints()
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(getTotalCharge()) + "\n"
        # add footer lines
        result += "Amount owed is " + str(self.getTotalCharge()) + "\n"
        result += "You earned " + str(self.getTotalfrequentRenterPoints) + " frequent renter points"
        return result
    def getTotalCharge(self):
        result = 0
        for rental in self._rentals:
            result += rental.getCharge()
        return result
    def getTotalfrequentRenterPoints(self):
        result = 0
        for rental in self._rentals:
            result += rental.getFrequentRenterPoints()
        return result

 - 这次重构增加了代码，并且增加了循环， 从而降低了性能，但这些是优化时候需要考量的问题
 - 此次重构后，所有的计算逻辑提炼出来， 为后续增加比如`htmlStatement`提供了便利
 
 ### 运用多态去掉与价格相关的条件逻辑
 
 > 最好不要在另一个对象的属性基础上运用switch语句，如果不得不使用也应该在对象自己的数据上使用，而不是别人的数据上使用
 
 - 因此 `getCharge`应该移动到`Movie`上去
 - 同理常客积分

In [10]:
# before
class Rental(object):
    def getCharge(self):
        result = 0
        if self.getMovie().getPriceCode() == 0:
            result += 2
            if self.getDaysRented() > 2:
                result += (self.getDaysRented() - 2) * 1.5
        elif self.getMovie().getPriceCode() == 1:
            result += self.getDaysRented() * 3
        else:
            result += 1.5
            if self.getDaysRented() > 3:
                result += (self.getDaysRented() - 3) * 1.5
        return result
# after
class Movie(object):
    def getCharge(self, daysRented):
        result = 0
        if self.PriceCode == Movie.REGULAR:
            result += 2
            if daysRented > 2:
                result += (daysRented - 2) * 1.5
        elif self.PriceCode == Movie.NEW_RELEASE:
            result += daysRented * 3
        else:
            result += 1.5
            if daysRented > 3:
                result += (daysRented - 3) * 1.5
        return result
    def getFrequentRenterPoints(self, daysRented):
        if (self.PriceCode == Movie.NEW_RELEASE) \
            and (daysRented > 1):
            return 2
        else:
            return 1  
class Rental(object):
    def getCharge(self):
        return self._movie.getCharge(self._daysRented)
    def getFrequentRenterPoints(self):
        return self._movie.getFrequentRenterPoints(_daysRented)

 - 为什么将租期长度传给Movie对象而不是将影片类型传给Rental对象呢？因为可能加入影片类型，尽量缩减其造成的影响
 
 ### 继承

- 可以为`Movie`建立不同影片的子类，这样就可以用多态来取代switch（if...elif...else）语句了,
不过这里有个小问题， 一部影片可以修改自己的分类，一个对象却不能修改自己的类型

```python
class Movie(object):
    pass
class RegularMovie(Movie):
    pass
class ChildrenMovie(Movie):
    pass
...
```

- 至此可以用四人帮（设计模式，Gang of Four）的**State模式**抽象出`Price`类用以计费

> State or Strategy? 取决于Price代表计费方式（Pricer or PriceStrategy）还是影片的某个状态，此时
> 对模式的选择反映了你对结构的想法

```python
class Movie(object):
    def getCharge(self, price):
        return price.getCharge()

class Price(object):
    pass

class RegularPrice(Price):
    pass

class ChildrenPrice(Price):
    pass

```
#### Replace Type Code with State/Strategy

- 把类型相关的行为搬移至State模式内，Move Method 将switch语句移至Price类，最后运用Replace Conditional with Polymorphism去掉switch语句